## You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb).

In [ ]:
!pip install deeppavlov

## Data Preparation

In [1]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

data = SimpleDSTC2DatasetReader().read('my_data')

2019-08-15 12:03:21.424 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 269: [downloading data from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data]
2019-08-15 12:03:21.452 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data/simple_dstc2.tar.gz
100%|██████████| 497k/497k [00:00<00:00, 47.4MB/s]
2019-08-15 12:03:21.466 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_data/simple_dstc2.tar.gz archive into my_data
2019-08-15 12:03:21.491 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading dialogs from my_data/simple-dstc2-trn.json]
2019-08-15 12:03:21.558 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading dialogs from my_data/simple-dstc2-val.json]
2019-08-15 12:03:21.607 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading di

In [2]:
!ls my_data

simple-dstc2-templates.txt  simple-dstc2-tst.json
simple-dstc2-trn.json	    simple-dstc2-val.json


In [5]:
!head -n 18 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },


In [6]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

In [7]:
from pprint import pprint

x_dialog, y_dialog = iterator.train[0]
pprint(x_dialog, indent=4)

[   {'prev_resp_act': None, 'text': ''},
    {'prev_resp_act': 'welcomemsg', 'text': 'cheap restaurant'},
    {'prev_resp_act': 'request_food', 'text': 'any'},
    {'prev_resp_act': 'request_area', 'text': 'south'},
    {   'db_result': {   'addr': 'cambridge leisure park clifton way cherry '
                                 'hinton',
                         'area': 'south',
                         'food': 'chinese',
                         'name': 'the lucky star',
                         'phone': '01223 244277',
                         'postcode': 'c.b 1, 7 d.y',
                         'pricerange': 'cheap'},
        'prev_resp_act': 'api_call',
        'text': 'south'},
    {'prev_resp_act': 'inform_area+inform_food+offer_name', 'text': 'address'},
    {'prev_resp_act': 'inform_addr+offer_name', 'text': 'phone number'},
    {'prev_resp_act': 'inform_phone+offer_name', 'text': 'thank you good bye'}]


## 1. Build database of items

In [8]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(table_name="mytable",
                           primary_keys=["name"],
                           keys=["name", "food", "pricerange", "area", "addr", "phone", "postcode"],
                           save_path="my_bot/db.sqlite")

2019-08-15 12:04:40.898 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-08-15 12:04:40.898 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 67: Initializing empty database on /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.


In [9]:
db_results = []
for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

2019-08-15 12:04:41.763 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 138: Created table with keys {'name': 'text', 'food': 'text', 'pricerange': 'text', 'area': 'text', 'addr': 'text', 'phone': 'text', 'postcode': 'text'}.


Adding 3016 items.


In [10]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'name': 'the lucky star',
   'food': 'chinese',
   'pricerange': 'cheap',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way cherry hinton',
   'phone': '01223 244277',
   'postcode': 'c.b 1, 7 d.y'},
  {'name': 'nandos',
   'food': 'portuguese',
   'pricerange': 'cheap',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way',
   'phone': '01223 327908',
   'postcode': 'c.b 1, 7 d.y'}]]

In [11]:
!ls my_bot

db.sqlite


## 2. Build Slot Filler

Slot Filler is component that inputs text and outputs dictionary of slot names and their values:

    slot_filler(['I would like some chineese food'])
    >> [{'food': 'chinese'}]

To implement a slot filler you need to provide
    
 - **slot types**
 - all possible **slot values**
 - optionally, it will be good to provide examples of mentions for every value of each slot
 
The data should be in `slot_vals.json` file with the following format:

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                

Let's use a simple non-trainable slot filler that relies on levenshtein distance:

### Non-trainable slot filler

In [12]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2019-08-15 12:04:45.398 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 10.9MB/s]
2019-08-15 12:04:45.400 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [13]:
!ls my_bot/slotfill

dstc_slot_vals.json


In [17]:
!head -n 10 my_bot/slotfill/dstc_slot_vals.json

{
    "food": {
        "caribbean": [
            "carraibean",
            "carribean",
            "caribbean"
        ],
        "kosher": [
            "kosher"
        ],


In [18]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json


slotfill_raw_config = read_json(configs.ner.slotfill_dstc2_raw)

In [19]:
slotfill_raw_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_raw_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

In [20]:
from deeppavlov import evaluate_model

slotfill_raw = evaluate_model(slotfill_raw_config);

2019-08-15 12:05:18.812 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 95: [downloading data from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/vimary/code-projects/Pilot/examples/my_data]
2019-08-15 12:05:18.819 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/vimary/code-projects/Pilot/examples/my_data/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 35.5MB/s]
2019-08-15 12:05:18.834 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting /home/vimary/code-projects/Pilot/examples/my_data/dstc2_v2.tar.gz archive into /home/vimary/code-projects/Pilot/examples/my_data
2019-08-15 12:05:18.856 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-15 12:05:18.998 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2

{"valid": {"eval_examples_count": 1444, "metrics": {"slots_accuracy": 0.9307}, "time_spent": "0:00:27"}}
{"test": {"eval_examples_count": 1386, "metrics": {"slots_accuracy": 0.9481}, "time_spent": "0:00:25"}}


We've got slot accuracy of **93% on valid** set and **94% on test** set.

In [21]:
from deeppavlov import build_model

slotfill_raw = build_model(slotfill_raw_config)

In [24]:
slotfill_raw(['i want cheap chinee food'])

[{'food': 'chinese', 'pricerange': 'cheap'}]

Saving slotfill config file to disk (we will require it's path later):

In [23]:
import json

json.dump(slotfill_raw_config, open('my_bot/slotfill_raw_config.json', 'wt'))

!ls my_bot

db.sqlite  slotfill  slotfill_raw_config.json


## Train bot

In [25]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_dstc2)

**Configure** bot to use our built **database**:

In [26]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'table_name': 'mytable',
    'primary_keys': ["name"],
    'keys': ["name", "food", "pricerange", "area", "addr", "phone", "postcode"],
    'save_path': 'my_bot/db.sqlite'
}

**Configure** bot to use levenshtein distance based **slot filler**:

In [27]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_raw_config.json'

gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = \
    ['pricerange', 'this', 'area', 'food']

**Configure** bot to use your **response templates**:

In [28]:
!head my_data/simple-dstc2-templates.txt -n 5

api_call	api_call area="#area" food="#food" pricerange="#pricerange"
bye	You are welcome!
canthear	Sorry, I can't hear you.
canthelp_area	I'm sorry but there is no #area american restaurant in the #area of town.
canthelp_area_food	Sorry there is no #food restaurant in the #area of town.


In [29]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['template_path'] = 'my_data/simple-dstc2-templates.txt'

You can use a simple **bag-of-words as embedder** (by default):

In [30]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

Specify train/valid/test **data path** and **path to save** the final bot model:

In [31]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

##### Training 

In [32]:
from deeppavlov import train_model

gobot_config['train']['max_batches'] = 120 # maximum number of training batches
gobot_config['train']['batch_size'] = 8 # set batch size
gobot_config['train']['val_every_n_batches'] = 30 # evaluate on 'valid' each 30 batches
gobot_config['train']['evaluation_targets'] = [] # do not evaluate after end of train (less time)

train_model(gobot_config);

2019-08-15 12:06:41.840 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-15 12:06:42.34 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-08-15 12:06:42.185 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
2019-08-15 12:06:42.493 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 98: [saving vocabulary to /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
W0815 12:06:43.502739 140072615057024 deprecation_wrapper.py:119] From /home/vimary/code-projects/Pilot/deeppavlov/core/models/tf_model.py:38: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0815 12:06:43.503

2019-08-15 12:06:45.605 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 243: [initializing `GoalOrientedBot` from scratch]
I0815 12:06:45.605098 140072615057024 network.py:243] [initializing `GoalOrientedBot` from scratch]
2019-08-15 12:08:06.88 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.0268
I0815 12:08:06.088352 140072615057024 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.0268
2019-08-15 12:08:06.89 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0815 12:08:06.089135 140072615057024 nn_trainer.py:166] Saving model
2019-08-15 12:08:06.90 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 12:08:06.090179 140072615057024 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
W0815 12:08:06.091090 140072615057024 deprecation_wrap

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0268}, "time_spent": "0:01:21", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 0.1667}, "time_spent": "0:01:36", "epochs_done": 0, "batches_seen": 15, "train_examples_seen": 120, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.9908068497975668}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 0.5128}, "time_spent": "0:01:50", "epochs_done": 0, "batches_seen": 30, "train_examples_seen": 240, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.4560860872268677}}


2019-08-15 12:09:55.443 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.4229
I0815 12:09:55.443168 140072615057024 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.4229
2019-08-15 12:09:55.443 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0815 12:09:55.443898 140072615057024 nn_trainer.py:166] Saving model
2019-08-15 12:09:55.444 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 12:09:55.444832 140072615057024 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-08-15 12:09:55.516 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 701: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0815 12:09:55.516048 140072615057024 network.py:701] [saving parameters to /home/vimary/code-projects/Pilot/examp

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4229}, "time_spent": "0:03:10", "epochs_done": 0, "batches_seen": 30, "train_examples_seen": 240, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 0.6667}, "time_spent": "0:03:25", "epochs_done": 0, "batches_seen": 45, "train_examples_seen": 360, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.181053606669108}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 0.5}, "time_spent": "0:03:40", "epochs_done": 0, "batches_seen": 60, "train_examples_seen": 480, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.9603077689806621}}


...

2019-08-15 12:13:31.784 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.4819
I0815 12:13:31.784088 140072615057024 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.4819
2019-08-15 12:13:31.784 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0815 12:13:31.784956 140072615057024 nn_trainer.py:166] Saving model
2019-08-15 12:13:31.785 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 12:13:31.785662 140072615057024 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-08-15 12:13:31.857 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 701: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0815 12:13:31.857128 140072615057024 network.py:701] [saving parameters to /home/vimary/code-projects/Pilot/examp

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4819}, "time_spent": "0:06:47", "epochs_done": 0, "batches_seen": 90, "train_examples_seen": 720, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 0.6452}, "time_spent": "0:07:01", "epochs_done": 0, "batches_seen": 105, "train_examples_seen": 840, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.6399111151695251}}
{"train": {"eval_examples_count": 8, "metrics": {"per_item_dialog_accuracy": 0.8022}, "time_spent": "0:07:16", "epochs_done": 0, "batches_seen": 120, "train_examples_seen": 960, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.5913254141807556}}


2019-08-15 12:15:19.815 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.5092
I0815 12:15:19.815698 140072615057024 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.5092
2019-08-15 12:15:19.816 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0815 12:15:19.816464 140072615057024 nn_trainer.py:166] Saving model
2019-08-15 12:15:19.817 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 12:15:19.817499 140072615057024 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-08-15 12:15:19.887 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 701: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0815 12:15:19.887879 140072615057024 network.py:701] [saving parameters to /home/vimary/code-projects/Pilot/examp

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.5092}, "time_spent": "0:08:35", "epochs_done": 0, "batches_seen": 120, "train_examples_seen": 960, "impatience": 0, "patience_limit": 10}}


2019-08-15 12:15:20.455 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 240: [initializing `GoalOrientedBot` from saved]
I0815 12:15:20.455870 140072615057024 network.py:240] [initializing `GoalOrientedBot` from saved]
2019-08-15 12:15:20.456 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 707: [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0815 12:15:20.456933 140072615057024 network.py:707] [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
2019-08-15 12:15:20.459 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 12:15:20.459510 140072615057024 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-08-15 12:15:20.500 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot

See [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) for advanced training configuration.

##### Metric scores on valid&test

In [110]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

2019-08-13 19:28:27.912 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
I0813 19:28:27.912971 139769053288064 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-13 19:28:28.17 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
I0813 19:28:28.017149 139769053288064 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-08-13 19:28:28.91 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
I0813 19:28:28.091510 139769053288064 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/e

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.5131}, "time_spent": "0:01:21"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.5059}, "time_spent": "0:01:19"}}


##### Chatting with bot

In [33]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2019-08-15 12:15:21.260 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
I0815 12:15:21.260463 140072615057024 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-08-15 12:15:21.263 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
W0815 12:15:21.263086 140072615057024 serializable.py:47] No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-08-15 12:15:21.264 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 63: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
I0815 12:15:21.264299 140072615057024 sqlite_database.py:63] Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-08-15 12:15:21.350 IN

In [63]:
bot(['hi, want chinee food, can you suggest a place to go?'])

2019-08-15 12:17:09.783 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 472: Made api_call with {'food': 'chinese'}, got 16 results.
I0815 12:17:09.783282 140072615057024 network.py:472] Made api_call with {'food': 'chinese'}, got 16 results.


['The lucky star serves chinese food.']

In [64]:
bot(['ok, i want ther address'])

['Sure, the lucky star is on cambridge leisure park clifton way cherry hinton.']

In [65]:
bot(['now i want french food in the center of town'])

2019-08-15 12:17:10.616 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 472: Made api_call with {'food': 'french', 'area': 'centre'}, got 1 results.
I0815 12:17:10.616809 140072615057024 network.py:472] Made api_call with {'food': 'french', 'area': 'centre'}, got 1 results.


['Cote is a nice place in the centre of town serving tasty french food.']

In [66]:
bot(['thanks, give me their address too'])

['Sure, cote is on bridge street city centre.']

In [67]:
bot(['and post code'])

['The post code of cote is c.b 2, 1 u.f.']

In [68]:
bot(['bye'])

['You are welcome!']

In [69]:
bot.reset()